In [1]:
def test(model, test_data, test_labels, difficulty):
    
    #-------------------------------------------description-------------------------------------------------------
    # model: .h5 file model type that will be used to make predictions
    # test_data: the data set that be used to make predictions from the given model (.npy file)
    # test_labels: labels of the provided dataset used to evaluate model (.npy file)
    # difficulty is a boolean input where a value of 1 will be for the hard data set and a value of 0 will be used 
    # for the easy data set
    # function will output the predicted labels and the accuracy
    # THIS WAS TESTED IN HPG Tensorflow-2.7.0 Kernel
    #-------------------------------------------------------------------------------------------------------------
    
    #loading test data
    import numpy as np
    import gc
    test_data = np.load(test_data)
    
    #reshaping array to unflatten it to a multidimensional array of size (# of images,imagesize,imagesize,3)
    total_photos = test_data.shape[1]
    img_size = 300
    reshaped_test = np.zeros((total_photos,img_size,img_size,3))

    for i in range(total_photos):
       reshaped_test[i] = np.reshape(test_data[:,0],(1,img_size,img_size,3))
       if i == total_photos - 1:
            del test_data
            gc.collect()
       else:
          test_data = test_data[:,1:]
    reshaped_test = reshaped_test.astype('int16')
    
    #resize to a smaller image
    import cv2

    new_size = 224
    resized_test = np.zeros((reshaped_test.shape[0],new_size,new_size,3))

    for i in range(total_photos):
        resized_test[i] = cv2.resize(reshaped_test[i],dsize = (new_size,new_size),interpolation = cv2.INTER_CUBIC)

    del reshaped_test
    gc.collect()
    
    #converting to 32bit float
    resized_test = resized_test.astype('float32')
    
    #loading in labels
    test_labels = np.load(test_labels)
    test_labels = test_labels.astype('int16')
    
    #loading in model
    import tensorflow as tf
    model_test = tf.keras.models.load_model(model);
    
    #easy blind dataset
    if difficulty == 0:
        
        #predictions of dataset
        predictions = model_test.predict(resized_test);
        
        #outputs predicted labels
        pred_labels = np.argmax(predictions, axis = 1)
        
        #accuracy of model
        from sklearn.metrics import accuracy_score
        accuracy = accuracy_score(test_labels, pred_labels)
        
    #hard blind dataset
    elif difficulty == 1:
        #predictions of dataset
        predictions = model_test.predict(resized_test);
        
        #pre-allocating array
        pred_labels = np.zeros((resized_test.shape[0]))
        
        for i in range(predictions.shape[0]):
            #assign class as unknown if larget prediction value is under the threshold
            if (np.max(predictions[i]) <= .835): #threshold of 83.5%
                pred_labels[i] = -1
            else:
                pred_labels[i] = np.argmax(predictions[i])
        
        #accuracy of model
        from sklearn.metrics import accuracy_score
        accuracy = accuracy_score(test_labels, pred_labels)
        
    return accuracy, pred_labels

In [ ]:
#run test function here
accuracy, labels = test(#,#,#,#)